Global setup


In [352]:
n=4
k=2
q=41
eta=2
R.<x> = ZZ[]

Bob's setup

In [353]:
Ax = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
Ax  #public

[24*x^3 + 24*x^2 + 25*x + 35     32*x^3 + 8*x^2 + x + 36]
[ 29*x^3 + 26*x^2 + 32*x + 2  17*x^3 + 19*x^2 + 40*x + 2]

In [354]:
def binomial_centered():
    a1=randrange(0,2)
    a2=randrange(0,2)
    b1=randrange(0,2)
    b2=randrange(0,2)
#    return a1+a2-b1-b2
    return a1-b1
    return 0

In [355]:
sx = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
sx  #private

[x^2 + 1]
[   -x^2]

In [356]:
ex = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
ex #private

[x^3 + x^2 - x + 1]
[              x^3]

In [357]:
tx = matrix(((Ax*sx + ex) % (x^n+1) % q))
tx  #public

[8*x^3 + 24*x^2 + 32*x + 20]
[22*x^3 + 26*x^2 + 20*x - 5]

**Matrices for lattice attack**


In [358]:
def negacirculant(v):
    Nshift = matrix([[0, 0, 0, -1],             #Negacyclic shift matrix
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]])
    return sum([v[i]*Nshift^i for i in range(0, len(v))])

In [359]:
Ax

[24*x^3 + 24*x^2 + 25*x + 35     32*x^3 + 8*x^2 + x + 36]
[ 29*x^3 + 26*x^2 + 32*x + 2  17*x^3 + 19*x^2 + 40*x + 2]

In [360]:
A = block_matrix([[negacirculant(Ax[0,0].padded_list(4)), negacirculant(Ax[0,1].padded_list(4))], 
                  [negacirculant(Ax[1,0].padded_list(4)), negacirculant(Ax[1,1].padded_list(4))]])
A

[ 35 -24 -24 -25| 36 -32  -8  -1]
[ 25  35 -24 -24|  1  36 -32  -8]
[ 24  25  35 -24|  8   1  36 -32]
[ 24  24  25  35| 32   8   1  36]
[---------------+---------------]
[  2 -29 -26 -32|  2 -17 -19 -40]
[ 32   2 -29 -26| 40   2 -17 -19]
[ 26  32   2 -29| 19  40   2 -17]
[ 29  26  32   2| 17  19  40   2]

In [361]:
tx

[8*x^3 + 24*x^2 + 32*x + 20]
[22*x^3 + 26*x^2 + 20*x - 5]

In [362]:
t = block_matrix([[column_matrix(1, 4, tx[0,0].padded_list(4))],
                [column_matrix(1, 4, tx[1,0].padded_list(4))]])
t

[20]
[32]
[24]
[ 8]
[--]
[-5]
[20]
[26]
[22]

In [363]:
I8 = identity_matrix(8)
I8

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

**Eve's attack**

Eve knows $A$ so she can set up $M$:

In [364]:
M = block_matrix([[A.transpose()], [q*I8]])
M

[ 35  25  24  24   2  32  26  29]
[-24  35  25  24 -29   2  32  26]
[-24 -24  35  25 -26 -29   2  32]
[-25 -24 -24  35 -32 -26 -29   2]
[ 36   1   8  32   2  40  19  17]
[-32  36   1   8 -17   2  40  19]
[ -8 -32  36   1 -19 -17   2  40]
[ -1  -8 -32  36 -40 -19 -17   2]
[-------------------------------]
[ 41   0   0   0   0   0   0   0]
[  0  41   0   0   0   0   0   0]
[  0   0  41   0   0   0   0   0]
[  0   0   0  41   0   0   0   0]
[  0   0   0   0  41   0   0   0]
[  0   0   0   0   0  41   0   0]
[  0   0   0   0   0   0  41   0]
[  0   0   0   0   0   0   0  41]

In [365]:
B, Transform = M.LLL(transformation=true)
B

[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0]
[ 0  0 -1  0  0  0  0  0]
[-1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1]
[ 0  0  0  0 -1  0  0  0]
[ 0  1  0  0  0  0  0  0]
[ 0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0  1  0]

In [366]:
Transform

[   119148362543   -151049731431    185002633596   -455487849422    -73828177742    -78972991525    -65570051714    420624928190   -358921368005               0               0               0               0               0               0               0]
[  1100981003159  -1395763074620   1709502176763  -4208899381270   -682203426524   -729743670633   -605894867313   3886751320044  -3316584461335               5               0               0               0               0               0               0]
[  -232550731993    294815009303   -361083416887    889009555285    144095952384    154137468573    127977952844   -820964995721    700533562327               0               1               0               0               0               0               0]
[ 17040576666002 -21603104515339  26459042272532 -65143787567869 -10558892258935 -11294702570049  -9377816609334  60157699052828 -51332867343315              82               0               1               0               0  

In [367]:
Transform.determinant()

1

In [368]:
Transform*M == B

True

Solve $M\mathbf{x} = \mathbf{t}$

In [369]:
xvector = B.solve_left(t.transpose())
xvector

[  0   0   0   0   0   0   0   0 -24 -20  22   5  32  -8  20  26]

In [370]:
xhat = xvector.apply_map(round, R=ZZ)
xhat

[  0   0   0   0   0   0   0   0 -24 -20  22   5  32  -8  20  26]

In [371]:
xhat*Transform % q

[ 6  4  3 30  3 20 34 26 34 12  0  0  0  0  0  0]

In [372]:
that = xhat*B
that

[20 32 24  8 -5 20 26 22]

In [373]:
t.transpose()-that

[0 0 0 0 0 0 0 0]

Solve $B\hat{\mathbf{s}} = \hat{\mathbf{t}}$

In [374]:
shat = M.solve_left(that, check=true)
shat

[ 43170347092/71784273601  34963773604/71784273601 -16472552802/71784273601 -24386324333/71784273601 -15294437400/71784273601 -13099239250/71784273601  11816598749/71784273601  15188604885/71784273601                        0                        0                        0                        0                        0                        0                        0                        0]

In [375]:
shat % q

[ 6  4  3 30  3 20 34 26  0  0  0  0  0  0  0  0]

In [376]:
sx

[x^2 + 1]
[   -x^2]

In [377]:
s = matrix([[1], [1], [1], [0],[-1], [1], [0], [1]])
s

[ 1]
[ 1]
[ 1]
[ 0]
[-1]
[ 1]
[ 0]
[ 1]

In [379]:
s-shat.transpose()[0:8]

[ 28613926509/71784273601]
[ 36820499997/71784273601]
[ 88256826403/71784273601]
[ 24386324333/71784273601]
[-56489836201/71784273601]
[ 84883512851/71784273601]
[-11816598749/71784273601]
[ 56595668716/71784273601]

**Alternative:  embedding**  (Knospe, p. 280)

In [380]:
AZ = A.augment(q*I8)
AZ

[ 35 -24 -24 -25  36 -32  -8  -1  41   0   0   0   0   0   0   0]
[ 25  35 -24 -24   1  36 -32  -8   0  41   0   0   0   0   0   0]
[ 24  25  35 -24   8   1  36 -32   0   0  41   0   0   0   0   0]
[ 24  24  25  35  32   8   1  36   0   0   0  41   0   0   0   0]
[  2 -29 -26 -32   2 -17 -19 -40   0   0   0   0  41   0   0   0]
[ 32   2 -29 -26  40   2 -17 -19   0   0   0   0   0  41   0   0]
[ 26  32   2 -29  19  40   2 -17   0   0   0   0   0   0  41   0]
[ 29  26  32   2  17  19  40   2   0   0   0   0   0   0   0  41]

In [417]:
AZ.transpose().LLL().transpose()

[ 0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
[ 0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
[ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]

In [381]:
H = A.transpose().hermite_form(include_zero_rows=false).transpose()
H

[           1            0            0            0            0            0            0            0]
[           0            1            0            0            0            0            0            0]
[           0            0            1            0            0            0            0            0]
[           0            0            0            1            0            0            0            0]
[           0            0            0            0            1            0            0            0]
[           0            0            0            0            0            1            0            0]
[           1            4            3            2            3            0            5            0]
[ 67740208903 192201946096   9159094559 110193022648 100076256650  17309173683 138029711320 358921368005]

In [413]:
B = ((H.augment(t))).stack(vector([0,0,0,0,0,0,0,0,1])) % q
B

[ 1  0  0  0  0  0  0  0 20]
[ 0  1  0  0  0  0  0  0 32]
[ 0  0  1  0  0  0  0  0 24]
[ 0  0  0  1  0  0  0  0  8]
[ 0  0  0  0  1  0  0  0 36]
[ 0  0  0  0  0  1  0  0 20]
[ 1  4  3  2  3  0  5  0 26]
[ 6 33  9 30 22 36 36 18 22]
[ 0  0  0  0  0  0  0  0  1]

In [414]:
B.transpose().LLL().transpose()

[ 0 -1  0  1 -1  0  0  0  0]
[ 0  1  0 -1 -1 -1  1  0  0]
[ 0 -1  1 -1  1 -1  0  1  0]
[ 0  0  0  0  1 -1 -1  1 -1]
[ 0  0  0  0  0  0 -1  1  1]
[-1  0  0  0  0  0  0  0  0]
[ 0  0  0 -1  0  1 -1  0 -1]
[ 0  0  1  0  0  0 -1 -1  0]
[ 0  0  1  0  0  0  0  1 -1]

In [409]:
e = block_matrix([[column_matrix(1, 4, ex[0,0].padded_list(4))],
                [column_matrix(1, 4, ex[1,0].padded_list(4))]])
e

[ 1]
[-1]
[ 1]
[ 1]
[--]
[ 0]
[ 0]
[ 0]
[ 1]

Still doesn't work.  Is the error vector too large?  (Is any reasonable error vector too large?)